In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\fmrol\AppData\Local\Temp\ipykernel_13432\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\fmrol\Documents\Ironhack\Classes Material\Week 4\Day 2\Lab NNL\lab-natural-language-processing\data\kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [4]:
data.head(10)

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
5,sure -- bottom line - you need a special secur...,0
6,"Dear Sir,I am Engr. Ugo Nzego with the Enginee...",1
7,Abedin Huma <AbedinH@state.gov>Saturday Novemb...,0
8,There is an Oct 16th George Marshall event at ...,0
9,<P>1 25% for you as the account owner <BR>2 65...,1


### Let's divide the training and test set into two partitions

In [5]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
# Assuming 'data' is the full dataset you provided earlier
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

# Check the shape of the training and validation data
print(f"Training set shape: {data_train.shape}")
print(f"Validation set shape: {data_val.shape}")



Training set shape: (800, 2)
Validation set shape: (200, 2)


## Data Preprocessing

In [6]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [7]:
# Assuming we haven't preprocessed the text yet, let's clean the text first

import re
from bs4 import BeautifulSoup

# Function to clean HTML and special characters
def clean_text(text):
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)  # Remove JS/CSS
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)  # Remove comments
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove multiple spaces
    return text.lower()  # Convert to lowercase

# Apply the cleaning function to the 'text' column
data_train['preprocessed_text'] = data_train['text'].apply(clean_text)
data_val['preprocessed_text'] = data_val['text'].apply(clean_text)


C:\Users\fmrol\AppData\Local\Temp\ipykernel_13432\2561301674.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
C:\Users\fmrol\AppData\Local\Temp\ipykernel_13432\2561301674.py:10: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
C:\Users\fmrol\AppData\Local\Temp\ipykernel_13432\2561301674.py:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [8]:
def clean_text(text):
    # Remove special characters and numbers using regex
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove single characters using regex
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # Remove single characters from the start using regex
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text


## Now let's work on removing stopwords
Remove the stopwords.

In [9]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()  # Split text into words
    clean_words = [word for word in words if word.lower() not in stop_words]  # Filter out stopwords
    return ' '.join(clean_words)  # Join the words back into a cleaned sentence

# Example usage
cleaned_text = remove_stopwords("This is an example sentence to remove stopwords.")
print(cleaned_text)


example sentence remove stopwords.


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [10]:
# Your codefrom nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Download WordNet data
nltk.download('wordnet')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = word_tokenize(text)  # Break sentence into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word
    return ' '.join(lemmatized_words)

# Example usage
lemmatized_text = lemmatize_text("The leaves are falling and the children are running")
print(lemmatized_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fmrol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fmrol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The leaf are falling and the child are running


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# Function to get the top n words in a category (spam/ham)
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)  # Initialize with stopwords
    bag_of_words = vec.transform(corpus)  # Transform the text data into BoW matrix
    sum_words = bag_of_words.sum(axis=0)  # Sum up occurrences of each word
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n]

# Split the dataset into spam and ham
spam_messages = data[data['label'] == 1]['text']  # Filter spam messages
ham_messages = data[data['label'] == 0]['text']   # Filter ham messages

# Get the top 10 words for spam and ham
top_spam_words = get_top_n_words(spam_messages, 10)
top_ham_words = get_top_n_words(ham_messages, 10)

# Display the top words for spam and ham
top_spam_words, top_ham_words

([('2e', 1857),
  ('br', 1103),
  ('money', 979),
  ('2c', 923),
  ('account', 835),
  ('bank', 779),
  ('transaction', 542),
  ('business', 511),
  ('country', 476),
  ('fund', 474)],
 [('state', 137),
  ('pm', 113),
  ('president', 98),
  ('mr', 86),
  ('2010', 84),
  ('obama', 84),
  ('30', 81),
  ('percent', 77),
  ('secretary', 75),
  ('10', 71)])

## Extra features

In [12]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,regards mr nelson smithkindly reply me on my ...,1,0,86
535,I have not been able to reach oscar this am. W...,0,i have not been able to reach oscar this am we...,1,0,95
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,huma abedin bim checking with pat on the k wi...,1,0,133
557,I can have it announced here on Monday - can't...,0,i can have it announced here on monday cant today,1,0,49
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,bank of africaagence san pedro bp san pedro c...,1,1,1580


## How would work the Bag of Words with Count Vectorizer concept?

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer, removing English stopwords
vectorizer = CountVectorizer(stop_words='english')

# Fit the vectorizer on the preprocessed training text and transform the text into a matrix of token counts
X_train_counts = vectorizer.fit_transform(data_train['preprocessed_text'])

# Check the shape of the resulting matrix
print(f"Shape of the training data after CountVectorizer: {X_train_counts.shape}")



Shape of the training data after CountVectorizer: (800, 17473)


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer, removing English stopwords
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit the vectorizer on the preprocessed training text and transform the text into a TF-IDF matrix
X_train_tfidf = tfidf_vectorizer.fit_transform(data_train['preprocessed_text'])

# Check the shape of the resulting TF-IDF matrix
print(f"Shape of the training data after TF-IDF transformation: {X_train_tfidf.shape}")



Shape of the training data after TF-IDF transformation: (800, 17473)


## And the Train a Classifier?

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Initialize the Multinomial Naive Bayes model
model = MultinomialNB()

# Fit the model to the TF-IDF transformed training data
model.fit(X_train_tfidf, data_train['label'])

# Transform the validation data using the same TF-IDF vectorizer
X_val_tfidf = tfidf_vectorizer.transform(data_val['preprocessed_text'])

# Predict the labels for the validation set
y_pred = model.predict(X_val_tfidf)

# Calculate the accuracy of the model
accuracy = accuracy_score(data_val['label'], y_pred)
print(f"Accuracy of the Multinomial Naive Bayes classifier: {accuracy:.4f}")


Accuracy of the Multinomial Naive Bayes classifier: 0.9600


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [20]:
# Use CountVectorizer instead of TfidfVectorizer for Bag of Words representation
X_train_counts = vectorizer.fit_transform(data_train['preprocessed_text'])
X_val_counts = vectorizer.transform(data_val['preprocessed_text'])

# Train the model using Bag of Words representation
model.fit(X_train_counts, data_train['label'])

# Predict the labels for the validation set
y_pred_counts = model.predict(X_val_counts)

# Calculate the accuracy for Bag of Words
accuracy_counts = accuracy_score(data_val['label'], y_pred_counts)
print(f"Accuracy using Bag of Words: {accuracy_counts:.4f}")


Accuracy using Bag of Words: 0.9500
